In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as olt


## Neural Network Challenge Assignment


Data Set Information:

Features are extracted from electric current drive signals. The drive has intact and defective components. This results in 11 different classes with different conditions. Each condition has been measured several times by 12 different operating conditions, this means by different speeds, load moments and load forces. The current signals are measured with a current probe and an oscilloscope on two phases.

From UCI machine learning database: https://archive.ics.uci.edu/ml/datasets/Dataset+for+Sensorless+Drive+Diagnosis#


In [3]:

#generating the table column names
Feature_columnnnames=[]
for i in range(48):
    Feature_columnnnames.append("Feature"+str(i+1))
Feature_columnnnames.append("label")

data = pd.read_csv('Sensorless_drive_diagnosis.txt', sep=" ", header=None,names=Feature_columnnnames)
data.head()


,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,...,Feature40,Feature41,Feature42,Feature43,Feature44,Feature45,Feature46,Feature47,Feature48,label
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,1
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,1
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,1
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,1
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,1


In [4]:
data.shape

(58509, 49)

In [6]:
data.isna().sum().sum() #calc total number of nulls

0

In [7]:
#split into test and train
from sklearn.model_selection import train_test_split
X = data.drop('label',axis=1)
Y = data.label

X_train, X_test, y_train, y_test = train_test_split(X,Y,shuffle= True,test_size=.2)

In [40]:
# Import the model
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(100,),max_iter=500) #start with default, 1 layer with 100 perceptrons
mlp.fit(X_train, y_train)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [41]:
print(cross_val_score(mlp,X_train,y_train,cv=3).mean())
print(cross_val_score(mlp,X_test,y_test,cv=3).mean())

0.9533837443862997


/Users/megandibble/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.8982957383913787


Pretty good for a first try, model is only slightly overfitting. I will try to tune the number of layers and number of perceptrons. 

In [42]:
#increasing the number of layers, keep number of perceptrons the same
mlp2 = MLPClassifier(hidden_layer_sizes=(100,100),max_iter=500)
mlp2.fit(X_train, y_train)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [43]:
print(cross_val_score(mlp2,X_train,y_train,cv=3).mean())
print(cross_val_score(mlp2,X_test,y_test,cv=3).mean())

0.9663740729067914
0.8954849719966562


In [29]:
#try increasing number of neurons in layer
mlp3 = MLPClassifier(hidden_layer_sizes=(1000,),max_iter=500)
mlp3.fit(X_train,y_train)

0.866863585785958

In [46]:
print(cross_val_score(mlp3,X_train,y_train,cv=3).mean())
print(cross_val_score(mlp3,X_test,y_test,cv=3).mean())

0.9438356525197803
0.8830991514078956


In [31]:
#try two smaller layers
mlp4 = MLPClassifier(hidden_layer_sizes=(50,50),max_iter=500)
mlp4.fit(X_train, y_train)


0.8993290030187239

In [47]:
print(cross_val_score(mlp4,X_train,y_train,cv=3).mean())
print(cross_val_score(mlp4,X_test,y_test,cv=3).mean())

0.9536001479744888
0.8916288952317616


### Building a random forest model to compare classification scores

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,y_train)

cross_val_score(rf,X_test,y_test,cv=5).mean()

/Users/megandibble/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9923950081752876

In [23]:
cross_val_score(rf,X_train,y_train,cv=5).mean()

0.9978206998176326

Random forest outperformed the neural networks--there is no need to use a neural network on this data when a more interpretable and less complex method gives a better score. Even though there is a lot of data in this dataset, it is not complex data (like images) and thus does not really merit using an ANN model. I did not realize this when I was picking the dataset originally